<a href="https://colab.research.google.com/github/claredavies/ArtificialIntelligience/blob/main/ComputationalIntelligienceAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [206]:
import numpy
import pandas as pd
import random

In [207]:
from numpy.random import randint
from numpy.random import rand

# Inputs

Class Info

In [208]:
#Creating Series
class_series = pd.Series(['Class1', 'Class1', 'Class1', 'Class1','Class2', 'Class2', 'Class2', 'Class2','Class3','Class3'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'English', 'History', 'Physics','Maths', 'Physics'])
hours_series = pd.Series([3, 4, 5, 3, 4, 4, 5, 3, 3, 4])


#Creating Dictionary
frame = { 'Class': class_series, 'Subject': subject_series, 'Hours': hours_series }

#Creating Dataframe
class_subject_hours_dataframe = pd.DataFrame(frame)

#Printing dataframe
print(class_subject_hours_dataframe)

    Class    Subject  Hours
0  Class1      Maths      3
1  Class1    English      4
2  Class1  Chemistry      5
3  Class1    History      3
4  Class2      Maths      4
5  Class2    English      4
6  Class2    History      5
7  Class2    Physics      3
8  Class3      Maths      3
9  Class3    Physics      4


Define Timetable

In [209]:
startTime = 10
durationLesson = 1
lunchTime = 12
durationLunch = 1
endTime = 15

singleTimetable = []
daysOfWeek = ['Mon','Tue','Wed', 'Thur']

for day in daysOfWeek:
  for time in range(startTime, endTime, durationLesson):
    if time != lunchTime:
      singleTimetable.append([day, time])

print(singleTimetable)

[['Mon', 10], ['Mon', 11], ['Mon', 13], ['Mon', 14], ['Tue', 10], ['Tue', 11], ['Tue', 13], ['Tue', 14], ['Wed', 10], ['Wed', 11], ['Wed', 13], ['Wed', 14], ['Thur', 10], ['Thur', 11], ['Thur', 13], ['Thur', 14]]


Teacher Info

In [210]:
teacher_series = pd.Series(['ProfessorA', 'ProfessorA', 'ProfessorB', 'ProfessorB','ProfessorC', 'ProfessorC'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'Physics'])

#Creating Dictionary
frame = { 'Teacher': teacher_series, 'Subject': subject_series}

#Creating Dataframe
teacher_subject_dataframe = pd.DataFrame(frame)

teacher_subject_array = teacher_subject_dataframe.to_numpy()
#Printing dataframe
print(teacher_subject_array)

# dictionary generated for convenience

# random availability list 
aval_list = []
for i in range(len(singleTimetable)):
  aval_list.append(random.choice([True,False]))

teachers_dict = dict((t, {
                        'Avaliability' : aval_list,  # Maybe is just enought with the number of hours assignable per day
                        'Subjects'      : teacher_subject_dataframe.groupby('Teacher').get_group(str(t))['Subject'].tolist()
                        }
                    ) for t in teacher_subject_dataframe.Teacher.unique())

print(teachers_dict)

[['ProfessorA' 'Maths']
 ['ProfessorA' 'English']
 ['ProfessorB' 'Chemistry']
 ['ProfessorB' 'History']
 ['ProfessorC' 'Maths']
 ['ProfessorC' 'Physics']]
{'ProfessorA': {'Avaliability': [True, True, True, False, True, False, True, False, False, True, True, True, False, True, True, False], 'Subjects': ['Maths', 'English']}, 'ProfessorB': {'Avaliability': [True, True, True, False, True, False, True, False, False, True, True, True, False, True, True, False], 'Subjects': ['Chemistry', 'History']}, 'ProfessorC': {'Avaliability': [True, True, True, False, True, False, True, False, False, True, True, True, False, True, True, False], 'Subjects': ['Maths', 'Physics']}}


In [211]:
def create_timetables(class_subject_hours_dataframe, timetable) -> numpy.ndarray:
  """
    Generates an array of blank timetables ready to be assigned, will have one row for each different class and n
    columns depending of the slots avaliable per week 
  """ 
  all_timetables = []
  blank_timetable = [None] * len(timetable)
  number_class_timetables = class_subject_hours_dataframe.Class.nunique()  # (Class 1, Class 2, ...)

  for x in range(number_class_timetables):
    all_timetables.append(blank_timetable)

  # print(all_timetables)
  all_timetables_array = numpy.array(all_timetables)
  return all_timetables_array

In [212]:
def make_array_1_hour_class(class_mask):
  """
  Generates a list with the subject names for convenience

  if the mask is

          Class    Subject  Hours
    0  Class1      Maths      3
    1  Class1    English      4
    2  Class1  Chemistry      5
    3  Class1    History      3
  
  The output will be

  ['Maths', 'Maths', 'Maths', 'English', 'English', 'English', 'English', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'History', 'History', 'History']
  """
  list_classes_1hr = []
  for index, row in class_mask.iterrows():
    number_hours_per_class = row['Hours']
    for x in range(number_hours_per_class):
      list_classes_1hr.append(row['Subject'])
  list_finished = list_classes_1hr.copy()
  return list_finished

def assign_teacher(subject:str, teacher_subject_dataframe):
  """ 
      Return a possible teacher name that teaches that subject, 
      in case of having more than one capable teacher, it will be
      selected randomly.

      TODO: Could be added a constrain of hours for each teacher
  """
  candidates = []
  for t in teacher_subject_dataframe.Teacher.unique():
    if subject in teachers_dict[t]['Subjects']:
      candidates.append(t)

  if len(candidates)==0:
    return 'Non-Possible-Assignation'
  elif len(candidates)==1:
    return candidates[0]
  else:
    return random.choice(candidates)

def reasignation_population(population, class_subject_hours_dataframe, teacher_subject_dataframe, timetable):
    """In case of having two populations been identical, we will regenerate the first one to appear in the list"""
    
    if len(population) == 1:  # we will not check in the case of only having one sample of population
        return population
    
    modified = population.copy() # This list will contain the population generated by the algorithm with the applyed changes
    
    for i in range(len(population)):
        for j in range(len(population)):  # checking each of the population samples
            if i == j:
                pass
            else:
                # is it worth to have this O(n²) just for checking identical pops? xd
                if((population[i] == population[j]).all()):
                    # we will delete one of the identicals and generate it again
                    # the probabilities of having the same output again should not be
                    # really high
                    print(" i = ", i, " j = ", j)
                    print("pop i -> \n", population[i], "\n")
                    print("pop j -> \n", population[j], "\n")
                    modified.remove(population[i])
                    update = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, timetable, populationSize = 1)[0]
                    population[i]=update # update for not double check (this will not modify the original one)
                    modified.append(update)
                
    return modified


## Test duplication

In [213]:
test = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, populationSize = 2)
test[0] = test[1] # force duplication
test
reasignation_population(test, class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable)

 i =  0  j =  1
pop i -> 
 [[('History', 'ProfessorB') ('Chemistry', 'ProfessorB')
  ('Chemistry', 'ProfessorB') ('Maths', 'ProfessorA')
  ('Chemistry', 'ProfessorB') ('English', 'ProfessorA')
  ('Maths', 'ProfessorC') ('English', 'ProfessorA')
  ('Maths', 'ProfessorC') ('Chemistry', 'ProfessorB') None
  ('History', 'ProfessorB') None ('English', 'ProfessorA')
  ('English', 'ProfessorA') ('Chemistry', 'ProfessorB')]
 [None ('Physics', 'ProfessorC') ('English', 'ProfessorA')
  ('Maths', 'ProfessorA') ('Maths', 'ProfessorA')
  ('History', 'ProfessorB') ('English', 'ProfessorA')
  ('History', 'ProfessorB') ('Maths', 'ProfessorA')
  ('History', 'ProfessorB') ('History', 'ProfessorB')
  ('Physics', 'ProfessorC') ('History', 'ProfessorB')
  ('Maths', 'ProfessorC') ('English', 'ProfessorA')
  ('English', 'ProfessorA')]
 [None None None None ('Maths', 'ProfessorA') None None None
  ('Maths', 'ProfessorA') None None ('Maths', 'ProfessorA')
  ('Physics', 'ProfessorC') ('Physics', 'ProfessorC')
 

[array([[('History', 'ProfessorB'), ('Chemistry', 'ProfessorB'),
         ('Chemistry', 'ProfessorB'), ('Maths', 'ProfessorA'),
         ('Chemistry', 'ProfessorB'), ('English', 'ProfessorA'),
         ('Maths', 'ProfessorC'), ('English', 'ProfessorA'),
         ('Maths', 'ProfessorC'), ('Chemistry', 'ProfessorB'), None,
         ('History', 'ProfessorB'), None, ('English', 'ProfessorA'),
         ('English', 'ProfessorA'), ('Chemistry', 'ProfessorB')],
        [None, ('Physics', 'ProfessorC'), ('English', 'ProfessorA'),
         ('Maths', 'ProfessorA'), ('Maths', 'ProfessorA'),
         ('History', 'ProfessorB'), ('English', 'ProfessorA'),
         ('History', 'ProfessorB'), ('Maths', 'ProfessorA'),
         ('History', 'ProfessorB'), ('History', 'ProfessorB'),
         ('Physics', 'ProfessorC'), ('History', 'ProfessorB'),
         ('Maths', 'ProfessorC'), ('English', 'ProfessorA'),
         ('English', 'ProfessorA')],
        [None, None, None, None, ('Maths', 'ProfessorA'), None, No

# Genetic Algorithms Functions

In [214]:
def generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, timetable, populationSize):
  population = []
  class_names = class_subject_hours_dataframe.Class.unique()
  number_class_names = class_subject_hours_dataframe.Class.nunique()

  for a in range(populationSize):
    trackClass = 0
    all_timetables = create_timetables(class_subject_hours_dataframe, timetable)
    for class_name in class_names:
      mask = class_subject_hours_dataframe.loc[class_subject_hours_dataframe['Class'] == class_name]
      list_classes_1hr = make_array_1_hour_class(mask)

      list_indices_single_timetable = [None] * len(timetable)

      if(len(list_classes_1hr) > len(list_indices_single_timetable) ):
        print("Not enough hours in timetable for class hours set!!")
        return

      for i in range(len(list_indices_single_timetable)):
        list_indices_single_timetable[i] = i
    
      range_end_single_timetable = len(list_indices_single_timetable) - 1

      for j in range((len(list_classes_1hr) - 1)):
        random_num = random.randint(0, range_end_single_timetable)

        random_num_index_value = list_indices_single_timetable[random_num]

        all_timetables[trackClass][random_num_index_value] = (list_classes_1hr[j], assign_teacher(subject=list_classes_1hr[j], teacher_subject_dataframe=teacher_subject_dataframe))
    
        list_indices_single_timetable.pop(random_num)
        range_end_single_timetable = range_end_single_timetable - 1
    
      trackClass = trackClass + 1
    
    population.append(all_timetables)
    population = reasignation_population(population, class_subject_hours_dataframe, teacher_subject_dataframe, timetable)
  return population

In [215]:
# CHECK: We have to check as 2 different clashes if we have same subject the same hour
# or same profesor the same hour in different clases?

def calClashes(pop):
  num_timetable_slots = pop.shape[1]
  num_classes = pop.shape[0]
  clash = 0
  for i in range(num_timetable_slots):
    for j in range(1, num_classes):
      # Original function here
      # if(pop[(j-1)][i] == pop[j][i]):
      #   if(pop[(j-1)][i] != None):
      #    clash = clash + 1
      if(pop[(j-1)][i] != None and pop[(j)][i] != None):  # If we are not dealing with empty spaces
        if(pop[(j-1)][i] == pop[j][i]) or pop[(j-1)][i][1] == pop[j][i][1]: # check if we have the same class at the same time or one teacher is assigned in two classes at the same time
          clash = clash + 1
  return clash

In [216]:
def numOfBlockClass(schedules):
  """
  having the same subject from the same professor
  provides better fitness
  """
  numOfBlockClass = 0
  for i in range(len(schedules)):
    for j in range(len(schedules[i])-1):
      #If they are not none and they are exactly the same
      if schedules[i][j] and schedules[i][j+1] and (schedules[i][j] == schedules[i][j+1]):
        numOfBlockClass += 1
  return numOfBlockClass


In [263]:
def fitnessFunction(pop):
  """
  fitnessFunction will return a fitness list
  1. element: num of clashes
  2. element: num of block classes
  """
  arrPop = numpy.array(pop)
  num_representations = arrPop.shape[0]
  NoClashesList = [None] * num_representations
  numOfBlockClassList = [None] * num_representations

  # print("no representations:  " + str(num_representations))
  for i in range(num_representations):
    populationSelect = pop[i]
    noClashes = calClashes(populationSelect)
    NoClashesList[i] = noClashes
    noBlockClass = numOfBlockClass(populationSelect)
    numOfBlockClassList[i] = noBlockClass

  fitness = [] #fitness is a list 
  fitness.append(NoClashesList)
  fitness.append(numOfBlockClassList)
  return fitness

In [264]:
pop = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, 10)
fitnessFunction(pop)

[[5, 4, 5, 5, 7, 8, 7, 9, 5, 4], [3, 4, 5, 4, 5, 7, 5, 7, 3, 3]]

In [219]:
def recombine(parent1, parent2):
  parent1Array = numpy.array(parent1)
  parent2Array = numpy.array(parent2)

  recombinationResults = []

  noTimetables = parent2Array.shape[0]
  crossOverIndex = int(parent1Array.shape[1]/2)

  recombinedParent1Array = parent1Array.copy()
  recombinedParent2Array = parent2Array.copy()

  for i in range(noTimetables): 
    classRecombinationResults = []  
    parent1FirstHalf = parent1Array[i][0:(crossOverIndex-1)]
    parent1SecondHalf = parent1Array[i][(crossOverIndex-1):(parent1.shape[1])]
    parent2FirstHalf = parent2Array[i][0:(crossOverIndex-1)]
    parent2SecondHalf = parent2Array[i][(crossOverIndex-1):(parent2.shape[1])]

    indicesListNotMatching1stParent = []
    indicesListNotMatching2ndParent = []
    length2ndHalf = len(parent2SecondHalf)

    parent1OriginalSecondHalfCopy = parent1SecondHalf.copy()
    parent1ResultSecondHalf = parent2SecondHalf.copy()

    parent2OriginalSecondHalfCopy = parent2SecondHalf.copy()
    parent2ResultSecondHalf = parent1SecondHalf.copy()

    for j in range(length2ndHalf):
      if parent1ResultSecondHalf[j] not in parent1OriginalSecondHalfCopy:
        indicesListNotMatching1stParent.append(j)
      else:
        resultParent1 = numpy.where(parent1OriginalSecondHalfCopy == parent1ResultSecondHalf[j])
        parent1OriginalSecondHalfCopy = numpy.delete(parent1OriginalSecondHalfCopy, resultParent1[0][0])

    for k in range(len(parent1OriginalSecondHalfCopy)):
      parent1ResultSecondHalf[indicesListNotMatching1stParent[k]] = parent1OriginalSecondHalfCopy[k]

    for l in range(length2ndHalf):
      if parent2ResultSecondHalf[l] not in parent2OriginalSecondHalfCopy:
        indicesListNotMatching2ndParent.append(l)
      else:
        resultParent2 = numpy.where(parent2OriginalSecondHalfCopy == parent2ResultSecondHalf[l])
        parent2OriginalSecondHalfCopy = numpy.delete(parent2OriginalSecondHalfCopy, resultParent2[0][0])

    for m in range(len(parent2OriginalSecondHalfCopy)):
      parent2ResultSecondHalf[indicesListNotMatching2ndParent[m]] = parent2OriginalSecondHalfCopy[m]

    recombinedParent1 = numpy.concatenate((parent1FirstHalf, parent1ResultSecondHalf))
    recombinedParent2 = numpy.concatenate((parent2FirstHalf, parent2ResultSecondHalf))

    recombinedParent1Array[i] = recombinedParent1
    recombinedParent2Array[i] = recombinedParent2

  recombinationResults.append(recombinedParent1Array)
  recombinationResults.append(recombinedParent2Array)

  return recombinationResults

In [270]:
def mutation(recombinationResults):
  """ Generation of """
    
  randGen = random.randint(0, 5)
  numberMutationsWanted = 3

  if randGen > 3:
    #print("Mutation")
    for i in range(numberMutationsWanted):
      randStartInd = random.randint(0, len(recombinationResults[0][0])-1)
      randEndInd = random.randint(0, len(recombinationResults[0][0])-1)

      for i in range(len(recombinationResults[0])):
        #print("Start mutation at:  " + str(randStartInd))
        #print("End mutation at:  " + str(randEndInd))
        startIndexValueParent1 = recombinationResults[0][i][randStartInd]
        recombinationResults[0][i][randStartInd] = recombinationResults[0][i][randEndInd]
        recombinationResults[0][i][randEndInd] = startIndexValueParent1

        startIndexValueParent2 = recombinationResults[1][i][randStartInd]
        recombinationResults[1][i][randStartInd] = recombinationResults[1][i][randEndInd]
        recombinationResults[1][i][randEndInd] = recombinationResults[1][i][randStartInd]

  #else:
    #print("No mutation")

  return recombinationResults

In [229]:
liste = [0,1,2,3,4,5,6]
liste[3]

3

In [277]:
def evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize, num_generations):
  population = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize)

  for generation in range(num_generations):
    fitness = fitnessFunction(population)

    min_value = min(fitness[0])
    # create index list with the min clash values
    # among them choose the one with max num of block classes
    min_indexes = [i for i,x in enumerate(fitness[0]) if x == min_value]
    max_value = 0
    for i in min_indexes:
      max_value = max(fitness[1][i],max_value) #want to maximize num of block classes
    min_index = fitness[1].index(max_value)
    
    fitnessWithoutMin = fitness[0].copy()
    fitnessWithoutMin[min_index] = 1000
    second_min_value = min(fitnessWithoutMin)
    
    min_indexes = [i for i,x in enumerate(fitness[0]) if x == second_min_value]
    max_value = 0
    for i in min_indexes:
      max_value = max(fitness[1][i],max_value) #want to maximize num of block classes
    second_min_index = fitness[1].index(max_value)

    firstParent = population[min_index]
    secondParent =  population[second_min_index]

    # print("Parent 1:  " + str(firstParent))
    # print("Parent 2:  " + str(secondParent))

    recombinationParentResults = recombine(firstParent, secondParent)
    # print("Recombination results:  " + str(recombinationParentResults))

    recombinationParentResultsMutation = mutation(recombinationParentResults)
    # print("Mutation results:  " + str(recombinationParentResultsMutation))

    # make up new population
    population = []
    population.append(firstParent)
    population.append(secondParent)
    population.append(recombinationParentResults[0])
    population.append(recombinationParentResults[1])
    # print("Population going in:  " + str(population))

    # The best result in the current iteration.
    fitness = fitnessFunction(population)
    min_value_clash = min(fitness[0])
    min_indexes = [i for i,x in enumerate(fitness[0]) if x == min_value_clash]
    max_value_block = 0
    for i in min_indexes:
      max_value_block = max(fitness[1][i],max_value_block) #want to maximize num of block classes
    min_index = fitness[1].index(max_value_block)
    print("Best result : " + str(population[min_index]))
    print("Number of clashes:  " + str(min_value_clash) + " --- Number of block courses:  " + str(max_value_block))

# Execution

In [279]:
intial_population_size = 10
num_generations = 200

evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intial_population_size, num_generations)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Best result : [[('Chemistry', 'ProfessorB') ('Chemistry', 'ProfessorB')
  ('English', 'ProfessorA') ('Chemistry', 'ProfessorB') None
  ('Chemistry', 'ProfessorB') ('English', 'ProfessorA')
  ('Maths', 'ProfessorC') ('Maths', 'ProfessorC')
  ('English', 'ProfessorA') ('Chemistry', 'ProfessorB') None
  ('History', 'ProfessorB') ('Maths', 'ProfessorC')
  ('English', 'ProfessorA') ('History', 'ProfessorB')]
 [('History', 'ProfessorB') ('History', 'ProfessorB')
  ('English', 'ProfessorA') ('Physics', 'ProfessorC')
  ('English', 'ProfessorA') ('Maths', 'ProfessorA')
  ('History', 'ProfessorB') ('English', 'ProfessorA') None
  ('Maths', 'ProfessorA') ('History', 'ProfessorB')
  ('History', 'ProfessorB') ('Physics', 'ProfessorC')
  ('English', 'ProfessorA') ('Maths', 'ProfessorC')
  ('Maths', 'ProfessorC')]
 [None ('Maths', 'ProfessorC') ('Physics', 'ProfessorC') None
  ('Maths', 'ProfessorC') None ('Physics', 'ProfessorC')
  ('Maths', 'ProfessorA') None None ('Physics', 'ProfessorC') None Non